In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

try:
    model = AutoModelForCausalLM.from_pretrained('openai-community/gpt2')
    tokenizer = AutoTokenizer.from_pretrained('openai-community/gpt2')
    generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
except Exception as e:
    print(f"Error initializing model: {e}")
    exit()

In [2]:
from langchain import hub
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader

# Create vector store and QA chain
try:
    document_loader = DirectoryLoader("C:/Users/Asus/Documents/MMAI/RBC_capstone/data_sources")
    documents = document_loader.load()
    
    embeddings = HuggingFaceEmbeddings()
    vector_store = Chroma.from_documents(documents, embeddings)
    
    retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
    llm = generator
    retriever = vector_store.as_retriever()
    combine_docs_chain = create_stuff_documents_chain(
        llm, retrieval_qa_chat_prompt
    )
    qa_chain = create_retrieval_chain(retriever, combine_docs_chain)
except Exception as e:
    print(f"Error creating QA chain: {e}")
    exit()

In [16]:
def chatbot():
    print("Chatbot: Hello! How can I help you today?")
    while True:
        user_input = input()
        if user_input.lower() in ["exit", "quit"]:
            print("Chatbot: Goodbye!")
            break
        try:
            # Assuming you need to pass user_input as part of a dictionary
            response = qa_chain.invoke({"input": user_input})  # Adjust as per documentation
            response_str = str(response)
            print(f"Chatbot: {response_str}")
        except Exception as e:
            print(f"Chatbot: Sorry, I encountered an error: {e}")

chatbot()

# checking the output .invoke format 

Chatbot: Hello! How can I help you today?
Chatbot: Sorry, I encountered an error: can only concatenate str (not "ChatPromptValue") to str
Chatbot: Goodbye!


In [21]:
query ="What specific characteristics must an asset have to be eligible as High-Quality Liquid Assets under OSFI's guidelines?"
response = qa_chain.invoke({"input": query})  # Adjust as per documentation
response_str = str(response)
print(f"Chatbot: {response_str}")

TypeError: can only concatenate str (not "ChatPromptValue") to str